In [1]:
import hashlib

class CustomMap:
    def __init__(self):
        self.__values = []
        self.__hash_map = {}

    def __repr__(self):
        return str(self.__hash_map)

    def generate_fingerprint(self, key):
        key_str = str(key)
        sha = hashlib.sha384()
        sha.update(key_str.encode('utf-8'))
        return sha.hexdigest()

    def __getitem__(self, key):
        hash_value = self.generate_fingerprint(key)
        if hash_value in self.__hash_map:
            index = self.__hash_map[hash_value]
            return self.__values[index]
        raise KeyError(f"Key {key} not found")

    def __setitem__(self, key, value):
        hash_value = self.generate_fingerprint(key)
        if hash_value in self.__hash_map:
            index = self.__hash_map[hash_value]
            self.__values[index] = value
        else:
            self.__hash_map[hash_value] = len(self.__values)
            self.__values.append(value)

    def __delitem__(self, key):
        hash_value = self.generate_fingerprint(key)
        if hash_value in self.__hash_map:
            index = self.__hash_map.pop(hash_value)
            self.__values.pop(index)
            for i in range(index, len(self.__values)):
                key = self.__values[i]
                self.__hash_map[self.generate_fingerprint(key)] = i
        else:
            raise KeyError(f"Key {key} not found")

    def __contains__(self, key):
        hash_value = self.generate_fingerprint(key)
        return hash_value in self.__hash_map

    def clear(self):
        self.__values.clear()
        self.__hash_map.clear()

    def copy(self):
        new_dict = CustomMap()
        new_dict.__values = self.__values.copy()
        new_dict.__hash_map = self.__hash_map.copy()
        return new_dict

    @classmethod
    def fromkeys(cls, seq, value=None):
        new_dict = cls()
        for key in seq:
            new_dict[key] = value
        return new_dict

    def get(self, key, default=None):
        hash_value = self.generate_fingerprint(key)
        if hash_value in self.__hash_map:
            index = self.__hash_map[hash_value]
            return self.__values[index]
        return default

    def items(self):
        return list(zip(self.__hash_map.keys(), self.__values))

    def keys(self):
        return list(self.__hash_map.keys())

    def pop(self, key, default=None):
        hash_value = self.generate_fingerprint(key)
        if hash_value in self.__hash_map:
            index = self.__hash_map.pop(hash_value)
            value = self.__values.pop(index)
            for i in range(index, len(self.__values)):
                key = self.__values[i]
                self.__hash_map[self.generate_fingerprint(key)] = i
            return value
        if default is not None:
            return default
        raise KeyError(f"Key {key} not found")

    def popitem(self):
        if not self.__hash_map:
            raise KeyError("popitem(): dictionary is empty")
        key, index = self.__hash_map.popitem()
        value = self.__values.pop(index)
        for i in range(len(self.__values)):
            self.__hash_map[self.generate_fingerprint(self.__values[i])] = i
        return key, value

    def setdefault(self, key, default=None):
        if key not in self:
            self[key] = default
        return self[key]

    def update(self, other):
        if isinstance(other, dict):
            for key, value in other.items():
                self[key] = value
        elif hasattr(other, 'items'):
            for key, value in other.items():
                self[key] = value

    def values(self):
        return self.__values

# Создание экземпляра
my_dict = CustomMap()

# __setitem__ и __getitem__
my_dict["carrot"] = 101
my_dict["lettuce"] = 202
print(my_dict["carrot"])    # 101
print(my_dict["lettuce"])   # 202

# __contains__
print("carrot" in my_dict)     # True
print("radish" in my_dict)    # False

# get
print(my_dict.get("carrot"))         # 101
print(my_dict.get("radish"))        # None
print(my_dict.get("radish", 0))     # 0

# setdefault
print(my_dict.setdefault("lettuce", 505))  # 202 (не меняется, уже есть)
print(my_dict.setdefault("radish", 505))  # 505 (создана новая)
print(my_dict["radish"])                 # 505

# pop
print(my_dict.pop("lettuce"))       # 202
print(my_dict.pop("lettuce", 9999))  # 9999 (уже удалён)

# popitem
my_dict["alpha"] = 11
my_dict["beta"] = 22
k, v = my_dict.popitem()
print(f"Popped: {k} -> {v}")       # Что-то из словаря

# keys
print("Keys:", my_dict.keys())     # Список всех хэшей (ключей)

# values
print("Values:", my_dict.values())  # Список всех значений

# items
print("Items:", my_dict.items())    # Список пар (хэш, значение)

# clear
my_dict.clear()
print("After clear:", my_dict.items())  # []

# fromkeys
md2 = CustomMap.fromkeys(["uno", "dos", "tres"], 4242)
print("fromkeys:", md2.items())   # Все значения — 4242

# copy
md3 = md2.copy()
print("copy:", md3.items())       # Должны быть те же, что и в md2

# update
md3.update({"uno": 1001, "cuatro": 4004})
print("after update:", md3.items())  # "uno" обновился, "cuatro" добавлен

# Проверка совместимости с dict.items()
d = {"zeta": 11, "omega": 22}
my_dict.update(d)
print("Merged dict:", my_dict.items())

101
202
True
False
101
None
0
202
505
505
202
9999
Popped: 2188a34dce44467232d84580cb4ef3eac659fa8ff1b008bd63954cd57174465ab2c7bdfd4a39454b19a72bbb549b855b -> 22
Keys: ['ef80ae902340ae063f7ddc5931b5811d39205df13d9784980cc7749acf49de117041ff49e6f129165dcd098454a6e219', 'aababcbf313be76c522ab6aab27c585ddf461a6b4c9c409fa08f8e4f555ddf506fd6ea3466dbc4b1114da9887fdcfbcc', '116267d3a97763d92d0fed8929ec7a1676fff217939efeb2a1211629baea3ee521989a76786bcb2bb2e42e9f06316d2e', '9cc3c0f06e170b14d7c52a8cbfc31bf9e4cc491e2aa9b79a385bcffa62f6bc619fcc95b5c1eb933dfad9c281c77208af', '68d19122d8381bbf1a86b0ea317313ef72e12b39bb566e9f59f7c6fc11ac001d72d699f66fec2d5a14a91bb076716561', '9b20aa6472eef4fd186d231637b1c1d55a5a434cc9130d6afcaaf486253a20c23a4eaeea419594c17f46bc53c7cee12e']
Values: [101, 505, 11]
Items: [('ef80ae902340ae063f7ddc5931b5811d39205df13d9784980cc7749acf49de117041ff49e6f129165dcd098454a6e219', 101), ('aababcbf313be76c522ab6aab27c585ddf461a6b4c9c409fa08f8e4f555ddf506fd6ea3466dbc4b1114da9887fd